In [31]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [32]:
train_df=pd.read_csv('/kaggle/input/home-credit-default-risk/application_train.csv')
train_df.head(20)

In [4]:
print("number of rows and colums")
print(train_df.shape[0]," ",train_df.shape[1])

In [33]:
lst=train_df.isnull().sum()
cnt=0
delst=[]
for i in range(len(lst)):
    if lst[i] >90000:
        delst.append(i)
        cnt+=1
cnt
#50 columns have 1/3rd of values missing and hence can be dropped
train_df.drop(train_df.columns[delst],axis=1,inplace=True)

In [88]:
#for categorical values, fill with mode
cat_vals=train_df.select_dtypes('object').nunique().index.tolist()
train_df.select_dtypes('object').nunique()
for i in cat_vals:
    train_df[i].fillna(train_df[i].mode()[0], inplace=True)

In [41]:
#find all numeric data
num_cols = train_df.select_dtypes(include=np.number).columns.tolist()
len(num_cols)
#there are 61 columns with numeric data
df_num= train_df.select_dtypes(include=np.number) #make a seperate dataset with them

In [29]:
df_num.describe()

In [43]:
train_df["TARGET"].value_counts()
# 0 means loan repayed, 1 means unpaid
#data is highly unbalanced

In [48]:
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns

In [89]:
#see comparision of defaulters vs non based on different categorical attributes
def show_category_by_target(df, columns):
    for column in columns:
        chart = sns.catplot(x=column, col='TARGET', data=df, kind='count')
        chart.set_xticklabels(rotation=65)

show_category_by_target(train_df, cat_vals)

In [52]:
#since the propotion of defaulters in the dataset is less, we cannot conclude anything definite, but it can be noticed that there is same trend/distribution of defaulters and non defaulters in all attributes.

For numeric data


In [55]:
#find age of the ppl in dataset
plt.hist(train_df['DAYS_BIRTH']/-365, edgecolor='k',bins=25)
plt.title('Age of Client');
plt.xlabel('Age (years)');
plt.ylabel('Count');
#shows there is almost normal distribution of all age groups

In [63]:
age_data=train_df[['TARGET','DAYS_BIRTH']]
age_data['YEARS_BIRTH']=age_data['DAYS_BIRTH']/-365
age_data['bin'] = pd.cut(age_data['YEARS_BIRTH'], 11, include_lowest = True)
age_groups  = age_data.groupby('bin').mean()
plt.bar(age_groups.index.astype(str), 100*age_groups['TARGET'])

# Plot labeling
plt.xticks(rotation=75);
plt.xlabel('Age Group (years)');
plt.ylabel('Failur to Reapy(%)')
plt.title('Failure to Repay by Age Group');
#younger generation most likely to default and not replay loan

In [77]:
#density plots of all numerical variables
for column in df_num.columns:
    print(column)
    sns.kdeplot(x=df_num[column])
    plt.show()

In [78]:
#box plot of all numeric data to spot outliers
for column in df_num.columns:
    print(column)
    sns.boxplot(x=df_num[column])
    plt.show()

**filling of missing values**

In [34]:
l=train_df.columns[train_df.isna().any()].tolist()
l

In [35]:
train_df['AMT_ANNUITY'].fillna(value=train_df['AMT_ANNUITY'].mean(),inplace=True)

In [52]:
train_df['AMT_GOODS_PRICE'].fillna(value=train_df['AMT_ANNUITY'].median(),inplace=True)

In [53]:
train_df['NAME_TYPE_SUITE'].fillna(value=train_df['AMT_ANNUITY'].median(),inplace=True)

In [54]:
train_df['CNT_FAM_MEMBERS'].fillna(value=train_df['AMT_ANNUITY'].median(),inplace=True)

In [56]:
train_df['EXT_SOURCE_2'].fillna(value=train_df['EXT_SOURCE_2'].mean(),inplace=True)
train_df['EXT_SOURCE_3'].fillna(value=train_df['EXT_SOURCE_3'].mean(),inplace=True)
train_df['OBS_30_CNT_SOCIAL_CIRCLE'].fillna(value=train_df['OBS_30_CNT_SOCIAL_CIRCLE'].median(),inplace=True)
train_df['DEF_30_CNT_SOCIAL_CIRCLE'].fillna(value=train_df['DEF_30_CNT_SOCIAL_CIRCLE'].median(),inplace=True)
train_df['OBS_60_CNT_SOCIAL_CIRCLE'].fillna(value=train_df['OBS_60_CNT_SOCIAL_CIRCLE'].median(),inplace=True)
train_df['DEF_60_CNT_SOCIAL_CIRCLE'].fillna(value=train_df['DEF_60_CNT_SOCIAL_CIRCLE'].median(),inplace=True)
train_df['DAYS_LAST_PHONE_CHANGE'].fillna(value=0,inplace=True)
train_df['AMT_REQ_CREDIT_BUREAU_HOUR'].fillna(value=train_df['AMT_REQ_CREDIT_BUREAU_HOUR'].mean(),inplace=True)
train_df['AMT_REQ_CREDIT_BUREAU_DAY'].fillna(value=train_df['AMT_REQ_CREDIT_BUREAU_DAY'].mean(),inplace=True)
train_df['AMT_REQ_CREDIT_BUREAU_WEEK'].fillna(value=train_df['AMT_REQ_CREDIT_BUREAU_WEEK'].mean(),inplace=True)
train_df['AMT_REQ_CREDIT_BUREAU_MON'].fillna(value=train_df['AMT_REQ_CREDIT_BUREAU_MON'].mean(),inplace=True)
train_df['AMT_REQ_CREDIT_BUREAU_QRT'].fillna(value=train_df['AMT_REQ_CREDIT_BUREAU_QRT'].mean(),inplace=True)
train_df['AMT_REQ_CREDIT_BUREAU_YEAR'].fillna(value=train_df['AMT_REQ_CREDIT_BUREAU_YEAR'].mean(),inplace=True)

**PCA AND DATA REDUCTION**

In [58]:
df_num= train_df.select_dtypes(include=np.number) #make a seperate dataset with non null numeric values

In [67]:
df_num.shape #original shape

In [59]:
target_variable = df_num["TARGET"]
features_variable=df_num.drop("TARGET",axis=1)


In [60]:
from sklearn.preprocessing import StandardScaler
standarized_data = StandardScaler().fit_transform(features_variable)


We want the total variance to be 95%

In [68]:
from sklearn import decomposition
pca = decomposition.PCA(n_components = 0.95)

In [70]:
pca_data = pca.fit_transform(standarized_data)

In [71]:
pca_data.shape
#thus the total shape has reduced from 61 to 47 columns

In [78]:
pca_df = pd.DataFrame(data = pca_data)
pca_df

In [74]:
cat_cols = train_df.select_dtypes('object')
cat_cols

In [81]:
new_df=pd.concat([cat_cols, pca_df], axis=1, join='inner')
new_df.shape
#121-50=71
#71-13=53 columns

In [80]:
new_df.head(10)